In [22]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
from fake_headers import Headers

sites = pd.read_excel('Sites.xlsx')

pr_cy = 'https://a.pr-cy.ru/'
dct = {}
for url in sites['Сайт']: 
    try:
        header = Headers(headers=True)
        headers = header.generate()
        dct.setdefault(url, {'soup' : BeautifulSoup(urlopen(pr_cy + url)),
                            'frames' : ''})
    except:
        print(url)
    
    
for url in sites['Сайт']:
    try:
        dct[url]['frames'] = pd.read_html(pr_cy + url)
    except:
        continue
        
def find_domen_age(url):
    for div in url.find_all('div', class_='analysis-test__name'):
        if div.text == 'Возраст домена':
            next_div = div.find_next('div')
            return next_div.text
        
        
for key, value in dct.items():
    dct[key]['Возраст_домена'] = find_domen_age(value['soup'])
    
    
for key in dct:   
    for table in dct[key]['frames']:
        if all([i in table.columns for i in ['Данные',  'День', 'Неделя',  'Месяц']]):
            dct[key]['Посещения'] = table
        if 'Яндекс ключевые слова' in table.columns:
            dct[key]['Яндекс ключевые слова'] = table
        if 'Google ключевые слова' in table.columns:
            dct[key]['Google ключевые слова'] = table 
            
            
dct2 = {'Адреса' : [],
       'Возраст домена' : [],
       'Просмотры за месяц' : [],
       'Посетитилей за месяц' : [],
       'Яндекс ключевые слова' : [],
       'Google ключевые слова' : []}


for key in dct:
    try:
        dct2['Адреса'].append(key)
    except:
        dct2['Адреса'].append('')
    try:        
        dct2['Возраст домена'].append(dct[key]['Возраст_домена'])
    except:
        dct2['Возраст домена'].append('')       
    try:        
        dct2['Просмотры за месяц'].append(dct[key]['Посещения']['Месяц'][0])
    except:
        dct2['Просмотры за месяц'].append('')       
    try:    
        dct2['Посетитилей за месяц'].append(dct[key]['Посещения']['Месяц'][1])
    except:
        dct2['Посетитилей за месяц'].append('')       
    try:    
        dct2['Яндекс ключевые слова'].append(dct[key]['Яндекс ключевые слова']['Яндекс ключевые слова'].tolist())
    except:
        dct2['Яндекс ключевые слова'].append('')       
    try:    
        dct2['Google ключевые слова'].append(dct[key]['Google ключевые слова']['Google ключевые слова'].tolist())
    except:
        dct2['Google ключевые слова'].append('')         
    
    
data = pd.DataFrame(dct2)

data['Возраст домена'] = data['Возраст домена'].apply(lambda x: ''.join([i for i in str(x) if i.isalnum() or i == ' ']))

data['Возраст домена'] = data['Возраст домена'].apply(lambda x: '' if x == 'None' else x)

data.to_excel('Статистика по сайтам.xlsx')
            

In [11]:
dct

{}